Classificação câncer de mama.


In [26]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split # dividir a base de dados entre teste e treino
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import torch.nn as nn

In [27]:
import torch
torch.__version__

'2.0.1+cu118'

In [28]:
np.random.seed(123) # semente geradora aleatória, para gerar os mesmos numeros aleatorios, para recurso de divisão de base de dados
torch.manual_seed(123) # para os pesos terem os mesmo resultads quando executados

In [29]:
# Carregando as bases de dados
previsor = pd.read_csv('/content/entradas_breast.csv')
classificador = pd.read_csv('/content/saidas_breast.csv')

In [30]:
# Observando a base previsor
previsor.head()


,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,186.0000,275.0000,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,243.0000,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,173.00000
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,198.0000,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,205.0000,0.4000,0.1625,0.2364,0.07678


In [31]:
# Observando a base classificador / 0 benigno e 1 maligno
classificador

,0
0,0
1,0
2,0
3,0
4,0
...,...
564,0
565,0
566,0
567,0


In [32]:
# Observando numero de linhas e colunas
previsor.shape

(569, 30)

In [34]:
classificador.shape

(569, 1)

In [35]:
# Observando valores nulos
previsor.isna().any()

 radius_mean                False
 texture_mean               False
 perimeter_mean             False
 area_mean                  False
 smoothness_mean            False
 compactness_mean           False
 concavity_mean             False
concave_points_mean         False
 symmetry_mean              False
 fractal_dimension_mean     False
 radius_se                  False
 texture_se                 False
 perimeter_se               False
 area_se                    False
 smoothness_se              False
 compactness_se             False
 concavity_se               False
 concave_points_se          False
 symmetry_se                False
 fractal_dimension_se       False
 radius_worst               False
 texture_worst              False
 perimeter_worst            False
 area_worst                 False
 smoothness_worst           False
 compactness_worst          False
 concavity_worst            False
 concave_points_worst       False
 symmetry_worst             False
 fractal_dimen

In [36]:
# Observando a base classificador
classificador.isna().any()

0    False
dtype: bool

In [38]:
# Dividindo as bases entre treinamento e teste
previsor_treino, previsor_teste, classe_treino, classe_teste = train_test_split(previsor, classificador, test_size = 0.25)



In [39]:
# Observando os registros previsor_treino
previsor_treino.shape

(426, 30)

In [40]:
# Observando os registros classe_treino
classe_treino.shape

(426, 1)

In [41]:
# Obervando os registros para teste previsor_teste
previsor_teste.shape

(143, 30)

In [42]:
# Obervando os registros para teste classe_teste
classe_teste.shape

(143, 1)

In [43]:
# Transformando os dados para tensores para ficar no formato Pytorch
# Primeiramente do pandas para numpy
# Em seguida do numpy para pytorch
# Observando formato atual
type(previsor_treino)

pandas.core.frame.DataFrame

In [44]:
# Convertendo para o numpy
type(np.array(previsor_treino))

numpy.ndarray

In [46]:
# Convertendo para pytorch
previsor_treino = torch.tensor(np.array(previsor_treino), dtype=torch.float)
classe_treino = torch.tensor(np.array(classe_treino), dtype = torch.float)

In [47]:
# Observando se a conversão foi feita previsor_treino
type(previsor_treino)

torch.Tensor

In [48]:
# Observando se a conversão foi feita classe_treino
type(classe_treino)

torch.Tensor

In [49]:
# Criando um dataset para passar como parâmetro para o Pytorch e unir as 2 variáveis em uma só
dados = torch.utils.data.TensorDataset(previsor_treino, classe_treino)

In [54]:
# Fornecendo dataloader para que o Pytoch faça as interações
# Calcula e ajusta os pesos a cada 10 registros e mantem o histórico para não repetir os pesos
treino = torch.utils.data.DataLoader(dados, batch_size=10, shuffle=True)


In [56]:
# Criando o modelo
# 30 neurônios na camada de entrada porque temos 30 atributos previsores
# 2 camadas ocultas, duas com 16 neurônios. (entrada 30 + saída 1 /2) = aprox 16
# 1 camada de saída com 1 neurônio, pois é classificação binária 0 ou 1, benigno ou maligno
classificador = nn.Sequential(
    nn.Linear(in_features=30, out_features=16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Linear(16, 1),
    nn.Sigmoid())


In [57]:
# Observando os parâmetros do modelo de rede neural
classificador.parameters

<bound method Module.parameters of Sequential(
  (0): Linear(in_features=30, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=16, bias=True)
  (3): ReLU()
  (4): Linear(in_features=16, out_features=1, bias=True)
  (5): Sigmoid()
)>

In [58]:
# Criando a função de erro
errofunc = nn.BCELoss()

In [65]:
# criando o otimizador. lr * weigh_decay ajuda o algoritmo a chegar ao min global. weigh diminui o valor da lr para se aroximar mais do gradiente
otimizador = torch.optim.Adam(classificador.parameters(), lr=0.001, weight_decay=0.0001)

In [68]:
# Treinando o modelo
# O Pytorch nescessita de um loop para fazer o treinamento
for i in range(200): # rodando 200 vezes a atualização dos pesos
  running_loss = 0. # variável acumulativa de erro

  for j in treino: # novo loop com a variavel treino que contém os previsores e classificadores divididos
    inputs, labels = j # inputs são as entradas e labels são as saídas
    otimizador.zero_grad() # zera os valores acumulados para cacular o gradiente separado

    outputs = classificador(inputs) # saída passando as entradas para prever
    loss = errofunc(outputs, labels) # calculado a valor do erro( previsões, respostas reais)
    loss.backward()
    otimizador.step() # ajuste dos pesos

    running_loss += loss.item()
  print('Época %3d: perda %.5f' % (i+1, running_loss/len(treino)))

Época   1: perda 0.14993
Época   2: perda 0.16895
Época   3: perda 0.15087
Época   4: perda 0.13330
Época   5: perda 0.10984
Época   6: perda 0.12729
Época   7: perda 0.11854
Época   8: perda 0.16056
Época   9: perda 0.15616
Época  10: perda 0.17061
Época  11: perda 0.12586
Época  12: perda 0.11607
Época  13: perda 0.15158
Época  14: perda 0.19889
Época  15: perda 0.20564
Época  16: perda 0.12387
Época  17: perda 0.10574
Época  18: perda 0.14243
Época  19: perda 0.11714
Época  20: perda 0.20699
Época  21: perda 0.14712
Época  22: perda 0.15846
Época  23: perda 0.12235
Época  24: perda 0.16136
Época  25: perda 0.12674
Época  26: perda 0.12384
Época  27: perda 0.10076
Época  28: perda 0.11889
Época  29: perda 0.11832
Época  30: perda 0.11454
Época  31: perda 0.11280
Época  32: perda 0.12953
Época  33: perda 0.14206
Época  34: perda 0.16151
Época  35: perda 0.11797
Época  36: perda 0.09573
Época  37: perda 0.13311
Época  38: perda 0.11234
Época  39: perda 0.09354
Época  40: perda 0.10392


In [70]:
# Visualizando os pesos após o treinamento
# Criando nova variável e transformando em lista para facilitar a visualização
pesos = list(classificador.parameters())
pesos

[Parameter containing:
 tensor([[-7.0598e-01,  7.5450e-02, -5.7791e-01, -1.9443e-01,  1.2021e-01,
           2.2378e-02, -1.3475e-02, -9.3254e-02, -2.0912e-01,  5.3563e-01,
           7.2281e-02,  6.5518e-02,  7.7060e-02,  4.6885e-02,  2.3225e-02,
           6.4989e-02, -4.7081e-01,  3.6957e-02,  6.9681e-01,  1.1067e-01,
          -9.2448e-02,  2.6232e-01, -4.1479e-01,  1.6303e-01,  3.6242e-01,
          -7.8483e-02, -3.7933e-01, -1.4703e-01,  2.4095e-01,  6.1180e-01],
         [-3.6489e-10, -9.3782e-06, -2.3264e-03, -2.0793e-02,  1.2736e-38,
           1.8070e-38,  2.0519e-38,  5.9812e-38, -2.7022e-39, -3.0153e-39,
           6.1457e-39, -6.2645e-03, -2.0303e-02, -6.3507e-03,  1.9739e-38,
          -3.1332e-38,  4.4849e-39,  4.9943e-38,  1.1449e-38,  6.2463e-39,
           5.4479e-02, -5.4356e-05, -3.1562e-03, -2.3277e-02, -7.6859e-39,
          -1.9336e-39,  1.0608e-39, -1.3593e-38, -6.1814e-39, -5.9267e-40],
         [ 1.0756e-01,  2.5670e-02,  2.2476e-02,  4.2743e-02,  1.1118e-01,


In [71]:
# Avaliando o modelo com a base de teste
# Colocando a variave em modo de avaliação
classificador.eval()


Sequential(
  (0): Linear(in_features=30, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=16, bias=True)
  (3): ReLU()
  (4): Linear(in_features=16, out_features=1, bias=True)
  (5): Sigmoid()
)

In [72]:
# Covertendo a base de teste para o formato Pytorch
previsor_teste = torch.tensor(np.array(previsor_teste), dtype=torch.float)

In [73]:
# Observando se a conversão foi feita
type(previsor_teste)

torch.Tensor

In [74]:
# Fazendo as previsões
previsoes = classificador.forward(previsor_teste)

In [75]:
# Observando
previsoes

tensor([[9.9741e-01],
        [1.0000e+00],
        [8.4218e-02],
        [9.7943e-01],
        [2.9927e-05],
        [9.8188e-01],
        [7.9119e-01],
        [7.1297e-01],
        [9.0256e-01],
        [2.6069e-01],
        [9.7110e-01],
        [9.9996e-01],
        [1.2429e-16],
        [9.9931e-01],
        [9.9833e-01],
        [4.1849e-01],
        [9.9781e-01],
        [1.0000e+00],
        [5.3290e-01],
        [1.0000e+00],
        [7.5167e-17],
        [5.6386e-04],
        [6.2226e-01],
        [1.0000e+00],
        [9.7498e-01],
        [1.8501e-03],
        [8.2726e-01],
        [9.9997e-01],
        [3.5894e-07],
        [9.9960e-01],
        [4.1662e-01],
        [9.9975e-01],
        [6.0744e-01],
        [9.6328e-01],
        [1.5091e-01],
        [9.9765e-01],
        [5.4899e-01],
        [1.0000e+00],
        [9.9039e-01],
        [4.6857e-04],
        [3.3185e-03],
        [8.4707e-01],
        [2.2888e-07],
        [2.4530e-01],
        [3.5528e-06],
        [1

In [76]:
# Classificando, se maior que 0.5 será igual a 1 true, maligno
previsoes = np.array(previsoes > 0.5)
previsoes

array([[ True],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [

In [78]:
# Verificando a taxa de acerto da rede neural com as previsões e os registros reais
precisao = accuracy_score(classe_teste, previsoes)
precisao


0.9020979020979021